*YeeArt*

---

Este projeto tem como objetivo a comunicação com a lâmpada inteligente Xiaomi Yeelight. Nele, encontrará recursos e presets para interagir e experimentar com as diversas cores e flows que a lâmpada tem a oferecer. Navegue por:

1. Sequências de cores
2. Cores fixas
3. Reconhecimento de voz

In [0]:
from yeelight import *
from yeelight import transitions
from yeelight import Flow
import yeelight as yl
import speech_recognition as sr
import ipywidgets as wdg
import math
import time

In [0]:
bulb = None

In [0]:
################################################################################ Funcionalidades:
def encontra_ip_dispositivos():
    ips = list()
    for dispositivo in yl.discover_bulbs():
        ips.append(dispositivo['ip'])
    return ips

def escolhe_flow(preset):    
    flow = Flow(count=0,                                                        #Roda flow indefinidamente (até comando stop_flow() ou turn_off())
            transitions=preset)
    return flow

def executa_flow(bulb, flow):
    bulb.start_flow(flow)

def brilho(brightness=1):
    bulb.set_brightness(brightness)
        
def escada(span, teto):                                                         #Utilizar span máximo de 4*teto
    escadaria = list()
    for i in range(span):
        if i < teto:
            escadaria.append(i%teto+1)
        if i > teto and i < 2*teto:
            escadaria.append(((teto-1)-i)%teto+1)
        if i > 2*teto and i < 3*teto:
            escadaria.append(i%teto+1)
        if i > 3*teto and i < 4*teto:
            escadaria.append(((3*teto-1)-i)%teto+1)        
    return escadaria

def reconhece_voz():
    r = sr.Recognizer()
    mic = sr.Microphone()
    try:
        with mic as source:
            print("Iniciando reconhecimento...")
            r.adjust_for_ambient_noise(source)
            time.sleep(.250)
            print("Fale!")
            audio = r.listen(source)
            transcript = r.recognize_google(audio)
            return transcript.lower()
    except sr.UnknownValueError:
        print("Reconhecimento deu ruim...")
        return -1
    except sr.RequestError:
        print("API fora do ar, pablito...")
        return -1

def menu_dispositivos(dispositivos_na_rede):
    inicio = "Os dispositivos conectados em sua rede são:\n\n"
    for i, dispositivo in enumerate(dispositivos_na_rede):
        inicio += str(i+1) + ". " + str(dispositivo.get_properties()['name']) + "\n"
    print(inicio)
    print("Escolha o dispositivo que quer programar.")
    escolha = int(input("Entre com números de -1 a {}: (0 para nenhum dispositivo, -1 para renomear dispositivo)\n\n".format(len(dispositivos_na_rede))))
    if escolha >= 1 and escolha <= len(dispositivos_na_rede):
        return dispositivos_na_rede[escolha-1]
    elif escolha == -1:
        disp = int(input("Qual dispositivo deseja renomear?\n"))
        if disp >= 1 and disp <= len(dispositivos_na_rede):
            novo_nome = str(input("Defina um novo nome para o dispositivo:\n"))
            dispositivos_na_rede[disp-1].set_name(novo_nome)
            return dispositivos_na_rede[disp-1]
        else: print("Entre com um número condizente.")
        
'''def menu_flow(presets):
    print("Escolha um dos presets abaixo: (1 a {} ou -1 para sair)".format(len(presets)))
    for i, preset in enumerate(presets.keys()):
        if i + 1 >= 10:
            print(str(i + 1) + " --------------------------> "+preset.title())
        else:
            print(str(i + 1) + "  --------------------------> "+preset.title())'''

def menu_cores(presets):
    print("Escolha uma das cores abaixo: (1 a {} ou -1 para sair)".format(len(presets)))
    bulb.ensure_on()
    botoes = wdg.RadioButtons(
                              options=presets.keys(),
                              value='Vermelho',
                              description='Cor:',
                              disabled=False
                              ) 
    botoes.observe(menu_cores_handler)
    display(botoes)

def menu_cores_handler(botao):
  if botao['name'] == 'value':
      bulb.set_rgb(*statics[botao['new']])

def menu_flow(presets):
    print("Escolha um dos presets abaixo: (1 a {} ou -1 para sair)".format(len(presets)))
    botoes = wdg.RadioButtons(
                              options=presets.keys(),
                              value='Loucura',
                              description='Flow:',
                              disabled=False
                              )
    botoes.observe(menu_flow_handler)
    display(botoes)

def menu_flow_handler(botao):
    if botao['name'] == 'value':
        flow = flows[botao['new']]
        executa_flow(bulb, flow)

def menu_voz():
    comandos = {'fiat lux' : bulb.turn_on, 'good night' : bulb.turn_off, 'toggle' : bulb.toggle}

    reconhece = True
    while reconhece:
        resultado = reconhece_voz()
        if resultado != -1:
            if resultado in comandos:
                comandos[resultado]()
            else: reconhece = False
        else: reconhece = False


def menu_principal():
    global bulb
    if bulb is None:
        ips = encontra_ip_dispositivos()
        dispositivos_na_rede = [yl.Bulb(bulb, effect='smooth', duration=1000) for bulb in ips]
        bulb = menu_dispositivos(dispositivos_na_rede)
    if bulb is not None:
        wdg.interact(brilho, brightness=wdg.IntSlider(min=1, max=100, value=25, description='Brilho', continuous_update=False))
        print("Quer brincar de quê?")
        escolha = int(input("1. Sequências de cores\n2. Cores fixas\n3. Reconhecimento de voz\n4. Sair\n\n"))
        escolhas = {1 : 'Sequências de cores', 2 : 'Cores fixas', 3 : 'Reconhecimento de voz', 4 : 'Sair'}
        if escolha in escolhas.keys():
            if escolha == 1:
                menu_flow(flows)
            
            elif escolha == 2:
                menu_cores(statics)

            elif escolha == 3:
                menu_voz()
                
            else:
              print("Programa encerrado.")

        else:
            print("Opção indisponível.")
        #if escolha != escolhas.get('Sair'):
        #wdg.interact(brilho, brightness=wdg.IntSlider(min=1, max=100, value=25, description='Brilho', continuous_update=False)) 
        
################################################################################ Presets

def police3(duration=200, brightness=70):
    transitions = [
        RGBTransition(255, 0, 0, brightness=brightness, duration=duration),
        RGBTransition(200, 0, 0, brightness=1, duration=duration),
        RGBTransition(145, 0, 0, brightness=brightness, duration=duration),
        SleepTransition(duration=duration),
        RGBTransition(0, 0, 255, brightness=brightness, duration=duration),
        RGBTransition(0, 0, 200, brightness=1, duration=duration),
        RGBTransition(0, 0, 145, brightness=brightness, duration=duration),
        SleepTransition(duration=duration),
    ]
    return transitions

def tranquilao(duration_light=5000, duration_sleep=500, brightness=49):
    transitions = [
        RGBTransition(255, 233, 207, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(201, 170, 136, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(74, 112, 139, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(226, 231, 186, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(255, 0, 38, brightness=brightness, duration=duration_light)
    ]
    return transitions 

def psyched(duration_light=1229, duration_sleep=4500, brightness=27):
    transitions = [
        RGBTransition(255, 94,  27, brightness=brightness, duration=duration_light),
        RGBTransition(107, 255, 107, brightness=27, duration=690),
        RGBTransition(27,  151, 255, brightness=36, duration=duration_light),
        RGBTransition(107, 255, 80, brightness=brightness, duration=duration_light),
        SleepTransition(duration=duration_sleep),
        RGBTransition(255, 52,  224, brightness=brightness, duration=1500),
        RGBTransition(175, 119, 255, brightness=36, duration=duration_light),
        RGBTransition(228, 255, 163, brightness=27, duration=1000),
        RGBTransition(255, 0, 0, brightness=brightness, duration=2000)
    ]
    return transitions 

def parabens_meu_irmao(brightness=100, duration_light=1000):
    return [RGBTransition(210, 0, 2, brightness=brightness, duration=440),
            RGBTransition(243, 0, 2, brightness=brightness, duration=440),
            RGBTransition(254, 2, 0, brightness=brightness, duration=duration_light),
            RGBTransition(233, 2, 0, brightness=69, duration=duration_light),
            RGBTransition(209, 0, 2, brightness=69, duration=duration_light),
            RGBTransition(148, 0, 2, brightness=40, duration=duration_light),
            RGBTransition(111, 0, 0, brightness=10, duration=duration_light),
            RGBTransition(52, 0, 0, brightness=5, duration=440),
            RGBTransition(42, 0, 0, brightness=brightness, duration=440)]

def loucura_total(hue_max=359, saturation=100, duration=500):
    return [HSVTransition(hue, saturation=saturation, duration=duration) for hue in range(0, hue_max, math.ceil(hue_max/9))]

def loucura_absurda(hue_max=359, duration=250):
    transitions = []
    for hue in range(0, hue_max, math.ceil(hue_max/9)):
        sat = hue%100
        transitions.append(HSVTransition(hue, saturation=sat, duration=duration))
    return transitions

def come_down(duration):
    return [RGBTransition(0,195,111, brightness=1, duration=550),
            RGBTransition(0,180,132, brightness=1, duration=duration)]
                        
flow_presets = {
           'Loucura' : loucura_total(hue_max=342, saturation=94, duration=440),
           'PEIP' : yl.transitions.lsd(duration=2500, brightness=33),
           'BURABURABURA' : yl.transitions.police(duration=285, brightness=88),
           'BURABURA' : yl.transitions.police2(duration=300, brightness=74),
           'BURA' : police3(duration=150, brightness=30), 
           'Natal' : yl.transitions.christmas(duration=230, brightness=50),
           'DeltaDirac' : yl.transitions.pulse(236, 195, 39, duration=400, brightness=77),
           'Aleatorio' : yl.transitions.randomloop(duration=750, brightness=100, count=9),
           'Flashmono' : yl.transitions.strobe(), 
           'Flashmulti' : yl.transitions.strobe_color(),
           'Alarme' : yl.transitions.alarm(duration=300),
           'Disco' : yl.transitions.disco(bpm=101),
           'Chill' : tranquilao(duration_light=6000, duration_sleep=600, brightness=22),
           'Psyched' : psyched(duration_light=1445, duration_sleep=5000, brightness=17),
           'RubOn' : parabens_meu_irmao(brightness=69, duration_light=420),
           'Latenight' : come_down(duration=720)
               }

flows = {mode : escolhe_flow(flow) for mode, flow in flow_presets.items()}

statics = { 'Vermelho' :        (255,  0 ,  0),
            'Verde' :           ( 0 , 255,  0),
            'Azul' :            ( 0 ,  0 , 255),
            'Magenta' :         (255,  0 , 255),
            'Ciano' :           ( 0 , 255, 255),
            'Amarelo' :         (255, 255,  0),
            'Branco' :          (255, 255, 255)
          }

In [8]:
menu_principal()

Os dispositivos conectados em sua rede são:

1. Zimmer

Escolha o dispositivo que quer programar.
Entre com números de -1 a 1: (0 para nenhum dispositivo, -1 para renomear dispositivo)

1


interactive(children=(IntSlider(value=25, continuous_update=False, description='Brilho', min=1), Output()), _d…

Quer brincar de quê?
1. Sequências de cores
2. Cores fixas
3. Reconhecimento de voz
4. Sair

2
Escolha uma das cores abaixo: (1 a 7 ou -1 para sair)


RadioButtons(description='Cor:', options=('Vermelho', 'Verde', 'Azul', 'Magenta', 'Ciano', 'Amarelo', 'Branco'…

!pip install yeelight
!pip install SpeechRecognition
!pip install ipywidgets